In [3]:
import pandas as pd
import numpy as np
import re
import functools
import operator
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [4]:
df = pd.read_csv('data/lenta-ru-news.csv')

clear_df = pd.DataFrame({})
clear_df['text'] = df.text.str.lower().str.replace(r"(\n|\t|\xa0)", ' ').str.replace(r"\.\s+", r".").str.replace(
    r"(«|»|,|:|\(|\)|\[|\]|,)", "")

clear_df['text'] = clear_df['text'].str.split(".")
clear_df.dropna(inplace=True)
all_sentences = functools.reduce(operator.iconcat, clear_df['text'], [])
true_idx = [idx for idx, sent in enumerate(all_sentences) if sent is not '']
all_sentences = list(operator.itemgetter(*true_idx)(all_sentences))

need_words = ["прокурор", "следствие", "авария", "гражданин", "указ", "акция", "белка", "граф", "орган", "вид"]
need_sentences = []
for idx in tqdm(range(len(all_sentences))):
    if len(set(need_words) & set(all_sentences[idx].split())) != 0:
        need_sentences.append(all_sentences[idx])

100%|████████████████████████████████████████████████████████████████████| 8432428/8432428 [00:26<00:00, 315316.89it/s]


In [5]:
words_dict = dict()

In [6]:
import tensorflow as tf
import tensorflow_hub as hub

W0624 06:42:46.924900  6888 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [ ]:
start = 13_000
step = 500

for sent_idx in range(start, len(need_sentences), step):
    print(sent_idx)
    
    elmo_tf = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz", trainable=True)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sent_batch = need_sentences[sent_idx-step:sent_idx]
    vectors = sess.run(elmo_tf(sent_batch, signature="default", as_dict=True)["word_emb"])
    print(vectors.shape)

    for sentence in sent_batch:
        new_keys = set(sentence.split()) - set(words_dict.keys())
        words_dict.update(
            dict(zip(new_keys, len(new_keys) * [[]]))
        )

    for cur_sentence, cur_vector in tqdm(zip(sent_batch, vectors)):
        split_sentence = cur_sentence.split()
        for word in range(len(split_sentence)):
            words_dict[split_sentence[word]].append(cur_vector[word])

    for word in need_words:
        difference = []
        for i in range(len(words_dict[word])-1):
#             difference.append(np.linalg.norm(words_dict[word][i] - words_dict[word][i+1]))
            difference.append(
                cosine_similarity(words_dict[word][i].reshape(1, -1) - words_dict[word][i+1].reshape(1, -1))
            )

        print(word, np.std(difference))
        
    sess.close()

13000
Instructions for updating:
Colocations handled automatically by placer.


W0624 06:48:06.136638  6888 deprecation.py:323] From d:\python 3.7.2\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0624 06:48:11.037601  6888 saver.py:1483] Saver not created because there are no variables in the graph to restore


(500, 45, 512)


...

прокурор 0.29371268
следствие 0.39850917
авария 0.35357144
гражданин 0.4842082
указ 0.32085538
акция 0.47969958
белка 0.47366464
граф 0.49507576
орган 0.373546
вид 0.110288754
16500
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0624 07:00:44.162150  6888 saver.py:1483] Saver not created because there are no variables in the graph to restore


(500, 61, 512)


500it [00:00, 7442.46it/s]


прокурор 0.3130544
следствие 0.39777136
авария 0.3510055
гражданин 0.4867977
указ 0.32204825
акция 0.47973385
белка 0.48467118
граф 0.49430943
орган 0.3748848
вид 0.10248425
17000
